## Import all the libraries

In [1]:
import torchvision.transforms as transforms
import torchvision
import torch
import scipy
import torchvision.datasets as datasets
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import cv2
from skimage import io
from skimage.transform import resize
import os

## Data download and transform

In [13]:
# Define the transformation for data - 
transform = transforms.Compose([
    transforms.Resize((100, 100)),   # Resize images to the required size
    transforms.Grayscale(num_output_channels=3),  # Convert to grayscale if needed
    transforms.ToTensor(),           # Convert images to PyTorch tensors
])

# Load the Caltech 101 dataset
caltech101_data100 = torchvision.datasets.Caltech101(
    root = './data',                      # Directory where data will be saved
    #root='/content/drive/MyDrive',       # If working with the google drive
    download=True,                        # Download the dataset if not done
    transform=transform                   # Transform the data into required format
)

Files already downloaded and verified


## Task 0A

In [3]:
# Function to load images from the entire Caltech101 dataset
def load_caltech101_dataset(dataset_path, target_size=(100, 100)):
    images = []

    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            file_path = os.path.join(root, file)
            image = io.imread(file_path)

            # Resize the image to a consistent size
            image_resized = resize(image, target_size, anti_aliasing=True)

            # Flatten the resized image
            flattened_image = image_resized.flatten()

            images.append(flattened_image)

    # Find the maximum length of flattened images
    max_length = max(len(image) for image in images)

    # Zero-pad images to ensure a consistent length
    padded_images = [np.pad(image, (0, max_length - len(image))) for image in images]

    return np.array(padded_images)

# Specify the path to the Caltech101 dataset
caltech101_path = 'data/caltech101/101_ObjectCategories'

# Extract images from the entire dataset
caltech101_images = load_caltech101_dataset(caltech101_path)

# Normalize pixel values to be between 0 and 1
caltech101_images_normalized = caltech101_images / 255.0

# Perform PCA
pca = PCA()
pca.fit(caltech101_images_normalized)

# Print the explained variance ratio for each component
explained_variance_ratio = pca.explained_variance_ratio_
print("Explained Variance Ratio for each component:")
print(explained_variance_ratio)

# Compute and print the cumulative explained variance
cumulative_explained_variance = np.cumsum(explained_variance_ratio)
print("\nCumulative Explained Variance:")
print(cumulative_explained_variance)

# Find the number of components that explain a certain threshold of variance (e.g., 95%)
threshold_variance = 0.95
num_components_threshold = np.argmax(cumulative_explained_variance >= threshold_variance) + 1
print(f"\nNumber of components to explain {threshold_variance * 100}% variance: {num_components_threshold}")


Explained Variance Ratio for each component:
[3.83282416e-01 8.85222761e-02 5.59737478e-02 ... 6.87561679e-12
 4.17192587e-12 1.15429846e-29]

Cumulative Explained Variance:
[0.38328242 0.47180469 0.52777844 ... 1.         1.         1.        ]

Number of components to explain 95.0% variance: 696


## TASK 0B

In [17]:
import os
import cv2
import numpy as np

def load_caltech101_data(data_dir='./data/caltech101/101_ObjectCategories', target_size=(100, 100)):
    image_data = []
    labels = []
    
    for label_index, category in enumerate(os.listdir(data_dir)):
        category_path = os.path.join(data_dir, category)
        if os.path.isdir(category_path):
            for file_name in os.listdir(category_path):
                if file_name.startswith('image') and file_name.endswith('.jpg'):
                    image_path = os.path.join(category_path, file_name)
                    image = cv2.imread(image_path)
                    image = cv2.resize(image, target_size)
                    image_data.append(image)
                    labels.append(label_index)  # Assigning the label index to the image
    
    return np.array(image_data), np.array(labels)

In [18]:
caltech101_data, caltech101_labels = load_caltech101_data()

In [23]:
from sklearn.decomposition import PCA

def compute_inherent_dimensionality(images, n_components=0.95):
    # Flatten the images
    flattened_images = images.reshape(images.shape[0], -1)

    # Apply PCA
    pca = PCA(n_components=n_components)
    pca.fit(flattened_images)

    # Get the number of principal components
    num_components = pca.n_components_

    return num_components

# Iterate over unique labels and calculate inherent dimensionality
unique_labels = np.unique(caltech101_labels)
for label in unique_labels:
    label_indices = np.where(caltech101_labels == label)[0]
    label_images = caltech101_data[label_indices]

    inherent_dimensionality = compute_inherent_dimensionality(label_images)
    print(f"Label {label}: Inherent Dimensionality = {inherent_dimensionality}")

Label 0: Inherent Dimensionality = 25
Label 1: Inherent Dimensionality = 66
Label 2: Inherent Dimensionality = 31
Label 3: Inherent Dimensionality = 29
Label 4: Inherent Dimensionality = 60
Label 5: Inherent Dimensionality = 39
Label 6: Inherent Dimensionality = 21
Label 7: Inherent Dimensionality = 27
Label 8: Inherent Dimensionality = 68
Label 9: Inherent Dimensionality = 39
Label 10: Inherent Dimensionality = 52
Label 11: Inherent Dimensionality = 35
Label 12: Inherent Dimensionality = 40
Label 13: Inherent Dimensionality = 39
Label 14: Inherent Dimensionality = 24
Label 15: Inherent Dimensionality = 21
Label 16: Inherent Dimensionality = 51
Label 17: Inherent Dimensionality = 37
Label 18: Inherent Dimensionality = 270
Label 19: Inherent Dimensionality = 48
Label 20: Inherent Dimensionality = 33
Label 21: Inherent Dimensionality = 59
Label 22: Inherent Dimensionality = 58
Label 23: Inherent Dimensionality = 31
Label 24: Inherent Dimensionality = 27
Label 25: Inherent Dimensionality 